In [217]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [218]:
from pathlib import Path
import subprocess
from typing import Any, Dict, List

import fire
import pandas as pd
from scipy.stats import hmean
from hf_bench.benchmark import ResultsTableRow


def get_columns() -> List[str]:
    return ResultsTableRow.__annotations__.keys()


def list_tracked_files(dirpath: str) -> List[str]:
    # Run git ls-tree command and capture output
    cmd = ["git", "ls-tree", "-r", "HEAD", "--name-only", dirpath]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    # Split output into list of files
    files = result.stdout.strip().split("\n")
    # Filter out empty strings
    files = [f for f in files if f]
    return files


def list_staged_files(dirpath: str) -> List[str]:
    cmd = ["git", "diff", "--name-only", "--cached", "HEAD", dirpath]
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    files = result.stdout.strip().split("\n")
    files = [f for f in files if f]
    return files


def get_df_concat(dirpath: str) -> pd.DataFrame:
    """
    Get a dataframe of all the results in the given directory.
    """
    filepaths = list_tracked_files(dirpath)
    print(f"Found {len(filepaths)} tracked files in {dirpath}.")
    df_first = pd.read_csv(filepaths[0])
    column_dtypes: Dict[str, Any] = {
        "submission_id": str,
        **{col: df_first[col].dtype for col in get_columns()},
    }
    columns = ["submission_id"] + list(get_columns())
    df = pd.DataFrame(columns=columns).astype(column_dtypes)
    for f in filepaths:
        submission_id: str = Path(f).parent.stem
        df_new = pd.read_csv(f)
        df_new["drafter"] = df_new["drafter"].fillna("No Drafter (Autoregressive)")
        df_new["submission_id"] = submission_id
        df_new = df_new[columns]
        df = pd.concat([df, df_new])
    df.sort_values(
        by=columns,
        inplace=True,
    )
    return df


def get_df_concat_filtered(df_concat: pd.DataFrame, minimum_new_toks: int) -> pd.DataFrame:
    df_concat_filtered = df_concat.set_index(["target", "submission_id", "dataset_path", "dataset_name", "dataset_split"])
    df_low_new_toks = (df_concat_filtered[df_concat_filtered["new_toks"] < minimum_new_toks]
                   .set_index("example_id", append=True)
                   .sort_index())
    index_low_new_toks = df_low_new_toks.index.unique() # Multi-index (target, submission_id, dataset_path, dataset_name, dataset_split, example_id) for which new_toks < 64
    # Remove all the rows corresponding to these multi-indices from df_concat_filtered
    df_concat_filtered.set_index("example_id", inplace=True, append=True)
    df_concat_filtered = df_concat_filtered[~df_concat_filtered.index.isin(index_low_new_toks)]
    df_concat_filtered.reset_index(inplace=True)
    return df_concat_filtered


def get_df_summary_of_results(df_concat: pd.DataFrame) -> pd.DataFrame:
    df_concat.reset_index(drop=True, inplace=True)
    columns_for_index: List[str] = [
        "target",
        "submission_id",
        "dataset_path",
        "drafter",
        "temperature",
    ]
    df_concat.set_index(columns_for_index, inplace=True)
    example_id_nunique = df_concat["example_id"].groupby(columns_for_index).nunique()
    df_summary = example_id_nunique.to_frame()
    df_summary.rename(columns={"example_id": "example_id_nunique"}, inplace=True)
    df_mean_vals = df_concat.groupby(columns_for_index)[["new_toks", "ttft_ms"]].mean()
    df_hmean_vals = df_concat.groupby(columns_for_index)[
        ["tpot_ms", "out_toks_per_sec"]
    ].agg(hmean)
    df_summary = pd.concat([df_summary, df_mean_vals, df_hmean_vals], axis=1)
    # Add the speedups
    df_otps = df_summary[["out_toks_per_sec"]]
    df_otps.reset_index(level="drafter", inplace=True)
    mask_ar = df_otps["drafter"] == "No Drafter (Autoregressive)"
    df_ar_otps = df_otps[mask_ar]
    df_ar_otps.drop(columns=["drafter"], inplace=True)
    # Reset the index of both dataframes to make the division operation simpler
    df_otps_reset = df_otps.reset_index()
    df_ar_otps_reset = df_ar_otps.reset_index()
    # Merge the dataframes on the common index columns
    merge_cols = ["target", "dataset_path", "temperature", "submission_id"]
    df_merged = pd.merge(
        df_otps_reset, df_ar_otps_reset, on=merge_cols, suffixes=("", "_ar")
    )
    # Perform the division
    df_merged["speedup"] = (
        df_merged["out_toks_per_sec"] / df_merged["out_toks_per_sec_ar"]
    )
    # Set back the multi-index structure
    df_speedups = df_merged.set_index(merge_cols + ["drafter"])[["speedup"]]
    df_summary.reset_index(inplace=True)
    df_summary.set_index(
        ["target", "dataset_path", "drafter", "temperature", "submission_id"],
        inplace=True,
    )
    df_summary = df_summary.join(df_speedups)
    # Reorder the multi-index columns
    df_summary.reset_index(inplace=True)
    new_index = ["target", "dataset_path", "submission_id", "temperature", "drafter"]
    df_summary.set_index(new_index, inplace=True)
    df_summary.sort_index(level=new_index, inplace=True)
    return df_summary


def get_df_max_speedup(df_summary: pd.DataFrame) -> pd.DataFrame:
    df_summary.reset_index(inplace=True)
    df_max_speedup = df_summary.loc[
        df_summary.groupby(["target", "dataset_path", "submission_id", "temperature"])[
            "speedup"
        ].idxmax()
    ]
    df_max_speedup.rename(columns={"drafter": "drafter_of_max_speedup"}, inplace=True)
    df_max_speedup.set_index(
        [
            "target",
            "temperature",
            "dataset_path",
            "submission_id",
            "drafter_of_max_speedup",
        ],
        inplace=True,
    )
    df_max_speedup.sort_index(inplace=True)
    return df_max_speedup


dirpath = "/home/projects/dharel/nadavt/repos/hf-bench/benchmark_results"
print("Concatenating all the results CSVs into one dataframe...")
df_concat: pd.DataFrame = get_df_concat(dirpath)
# df_concat.to_csv("results_all.csv", index=False)

minimum_new_toks = 128
print(f"Filtering out experiments with less than {minimum_new_toks} new tokens...")
df_concat_filtered = get_df_concat_filtered(df_concat, minimum_new_toks)

print("Counting the number of unique example IDs for each experiment...")
df_summary: pd.DataFrame = get_df_summary_of_results(df_concat_filtered)
# Round the values to 1 decimal place
df_summary["new_toks"] = df_summary["new_toks"].round(1)
df_summary["ttft_ms"] = df_summary["ttft_ms"].round(1)
df_summary["tpot_ms"] = df_summary["tpot_ms"].round(1)
df_summary["out_toks_per_sec"] = df_summary["out_toks_per_sec"].round(1)
df_summary["speedup"] = df_summary["speedup"].round(2)
# df_summary.to_csv("results_summary.csv", index=True)

print("Getting the maximum speedup for each experiment...")
df_max_speedup: pd.DataFrame = get_df_max_speedup(df_summary.copy())
# df_max_speedup.to_csv("results_max_speedup.csv", index=True)

df_summary_deepseek_qwen_14b = df_summary[df_summary.index.get_level_values("target").str.startswith("deepseek-ai/DeepSeek-R1-Distill-Qwen-14B")]
df_summary_deepseek_qwen_14b_temperature_0 = df_summary_deepseek_qwen_14b[df_summary_deepseek_qwen_14b.index.get_level_values("temperature") == 0]

# print(f"Stored both the concatenated dataframe and the summary in {dirpath}.")
# print("Done!")

df_concat.head(5)

Concatenating all the results CSVs into one dataframe...
Found 48 tracked files in /home/projects/dharel/nadavt/repos/hf-bench/benchmark_results.
Filtering out experiments with less than 128 new tokens...
Counting the number of unique example IDs for each experiment...
Getting the maximum speedup for each experiment...


/tmp/ipykernel_3673706/2939367320.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ar_otps.drop(columns=["drafter"], inplace=True)


,submission_id,target,dataset_path,dataset_name,dataset_split,num_of_examples,drafter,temperature,example_id,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
0,2025-02-04_01-05-29_4c55336,deepseek-ai/DeepSeek-R1-Distill-Llama-70B,openai/openai_humaneval,openai_humaneval,test,30,No Drafter (Autoregressive),0,0,512,2016.254902,129.010985,7.751278
1,2025-02-04_01-05-29_4c55336,deepseek-ai/DeepSeek-R1-Distill-Llama-70B,openai/openai_humaneval,openai_humaneval,test,30,No Drafter (Autoregressive),0,1,512,236.413956,126.681199,7.893831
2,2025-02-04_01-05-29_4c55336,deepseek-ai/DeepSeek-R1-Distill-Llama-70B,openai/openai_humaneval,openai_humaneval,test,30,No Drafter (Autoregressive),0,2,512,215.317965,119.733867,8.351856
3,2025-02-04_01-05-29_4c55336,deepseek-ai/DeepSeek-R1-Distill-Llama-70B,openai/openai_humaneval,openai_humaneval,test,30,No Drafter (Autoregressive),0,3,512,306.704283,123.782173,8.078708
4,2025-02-04_01-05-29_4c55336,deepseek-ai/DeepSeek-R1-Distill-Llama-70B,openai/openai_humaneval,openai_humaneval,test,30,No Drafter (Autoregressive),0,4,512,296.703815,120.254724,8.315682


In [219]:
df_concat.describe()

,num_of_examples,temperature,example_id,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
count,10650.0,10650.000000,10650.000000,10650.000000,10650.000000,10650.000000,10650.000000
mean,30.0,0.498592,14.500000,393.104413,429.016698,54.531549,39.551700
std,0.0,0.500021,8.655848,153.698989,553.005558,44.652178,1563.060465
min,30.0,0.000000,0.000000,8.000000,20.835876,0.006199,0.300873
25%,30.0,0.000000,7.000000,254.000000,101.354957,32.555696,14.979037
50%,30.0,0.000000,14.500000,510.000000,212.974191,47.632325,20.994146
75%,30.0,1.000000,22.000000,512.000000,504.370451,66.759968,30.716591
max,30.0,1.000000,29.000000,512.000000,3398.638010,3323.662519,161319.384615


In [220]:
df_concat_filtered.describe()

,example_id,num_of_examples,new_toks,ttft_ms,tpot_ms,out_toks_per_sec
count,8945.000000,8945.0,8945.000000,8945.000000,8945.000000,8945.000000
mean,14.285187,30.0,432.308664,423.365627,53.570710,24.685422
std,8.691103,0.0,119.733231,553.718037,31.205392,13.158204
min,0.000000,30.0,128.000000,20.835876,10.357687,3.702622
25%,7.000000,30.0,374.000000,91.443539,32.012641,15.252676
50%,14.000000,30.0,512.000000,203.604698,47.616901,21.000947
75%,22.000000,30.0,512.000000,478.106737,65.562265,31.237660
max,29.000000,30.0,512.000000,3398.638010,270.078875,96.546653


In [221]:
df_summary.head(5)

example_id_nunique  \
target                                    dataset_path  submission_id               temperature drafter                                                         
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2025-02-04_20-52-14_eb664c5 0           No Drafter (Autoregressive)                                30   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                                    1           No Drafter (Autoregressive)                                30   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   

                                                                                                                                           new_toks  \
target                                    dataset_path  submission_id               temperature drafter                                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2025-02-04_20-52-14_eb664c5 0           No Drafter (Autoregressive)                   494.7   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B      492.7   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     496.6   
                                                                                    1           No Drafter (Autoregressive)                   472.6   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B      462.0   

                                                                                                                                           ttft_ms  \
target                                    dataset_path  submission_id               temperature drafter                                              
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2025-02-04_20-52-14_eb664c5 0           No Drafter (Autoregressive)                  232.3   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B     461.3   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B    296.1   
                                                                                    1           No Drafter (Autoregressive)                  232.6   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B     348.0   

                                                                                                                                           tpot_ms  \
target                                    dataset_path  submission_id               temperature drafter                                              
deepseek-ai/DeepSeek-R1-Distill-Llama-70B cnn_dailymail 2025-02-04_20-52-14_eb664c5 0           No Drafter (Autoregressive)                   77.4   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Llama-8B      79.3   
                                                                                                deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     72.6   
                                                                                    1           No Drafter (Autoregressive)                   77.7   
  

In [222]:
df_max_speedup.head(5)

example_id_nunique  \
target                                    temperature dataset_path            submission_id               drafter_of_max_speedup                                          
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2025-02-04_01-05-29_4c55336 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-06_16-28-26_ab73cc7 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   

                                                                                                                                                     new_toks  \
target                                    temperature dataset_path            submission_id               drafter_of_max_speedup                                
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     496.6   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                       493.1   
                                                      openai/openai_humaneval 2025-02-04_01-05-29_4c55336 deepseek-ai/DeepSeek-R1-Distill-Llama-8B      512.0   
                                                                              2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B      512.0   
                                                                              2025-02-06_16-28-26_ab73cc7 deepseek-ai/DeepSeek-R1-Distill-Llama-8B      512.0   

                                                                                                                                                     ttft_ms  \
target                                    temperature dataset_path            submission_id               drafter_of_max_speedup                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B    296.1   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                      242.3   
                                                      openai/openai_humaneval 2025-02-04_01-05-29_4c55336 deepseek-ai/DeepSeek-R1-Distill-Llama-8B     353.5   
                                                                              2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B     222.6   
                                                                              2025-02-06_16-28-26_ab73cc7 deepseek-ai/DeepSeek-R1-Distill-Llama-8B     224.3   

                                                                                                                                                     tpot_ms  \
target                                    temperature dataset_path            submission_id               drafter_of_max_speedup                               
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B     72.6   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                       51.4   
                                                      openai/o

In [223]:
mask_deepseek_target = df_max_speedup.index.get_level_values("target").str.startswith("deepseek")
df_deepseek = df_max_speedup[mask_deepseek_target]
df_deepseek

example_id_nunique  \
target                                    temperature dataset_path            submission_id               drafter_of_max_speedup                                          
deepseek-ai/DeepSeek-R1-Distill-Llama-70B 0           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2025-02-04_01-05-29_4c55336 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-06_16-28-26_ab73cc7 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                      tau/scrolls             2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                          1           cnn_dailymail           2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-06_17-46-21_ab73cc7 meta-llama/Llama-3.2-1B                                    29   
                                                      openai/openai_humaneval 2025-02-04_01-05-29_4c55336 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                                              2025-02-06_16-28-26_ab73cc7 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
                                                      tau/scrolls             2025-02-04_20-52-14_eb664c5 deepseek-ai/DeepSeek-R1-Distill-Llama-8B                   30   
deepseek-ai/DeepSeek-R1-Distill-Llama-8B  0           cnn_dailymail           2025-02-05_03-18-59_d7c02fb double7/vicuna-68m                                         30   
                                                                              2025-02-06_17-06-51_ab73cc7 double7/vicuna-68m                                         30   
                                                      openai/openai_humaneval 2025-02-06_02-07-38_ea3ed4b No Drafter (Autoregressive)                                30   
                                                                              2025-02-06_16-26-53_ab73cc7 No Drafter (Autoregressive)                                30   
                                                      tau/scrolls             2025-02-05_03-18-59_d7c02fb meta-llama/Llama-3.2-1B                                    30   
                                                                              2025-02-06_17-06-51_ab73cc7 meta-llama/Llama-3.2-1B                                    29   
                                          1           cnn_dailymail           2025-02-05_03-18-59_d7c02fb double7/vicuna-68m                                         30   
                                                                              2025-02-06_17-06-51_ab73cc7 double7/vicuna-68m                                         30   
                                                      openai/openai_humaneval 2025-02-06_02-07-38_ea3ed4b No Drafter (Autoregressive)                                30   
                                                                              2025-02-06_16-26-53_ab73cc7 No Drafter (Autoregressive)                                30   
                                             

In [224]:
df_summary_deepseek_qwen_14b_temperature_0

example_id_nunique  \
target                                   dataset_path            submission_id               temperature drafter                                                         
deepseek-ai/DeepSeek-R1-Distill-Qwen-14B cnn_dailymail           2025-02-06_02-10-28_d8c80fe 0           No Drafter (Autoregressive)                                16   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  16   
                                                                                                         double7/vicuna-68m                                         16   
                                                                 2025-02-06_06-54-21_ab73cc7 0           No Drafter (Autoregressive)                                18   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  18   
                                                                                                         double7/vicuna-68m                                         18   
                                         openai/openai_humaneval 2025-02-06_02-10-28_d8c80fe 0           No Drafter (Autoregressive)                                30   
                                                                                                         bigcode/tiny_starcoder_py                                  30   
                                                                                                         codellama/CodeLlama-7b-Instruct-hf                         30   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                 2025-02-06_06-29-56_17d3c65 0           No Drafter (Autoregressive)                                30   
                                                                                                         bigcode/tiny_starcoder_py                                  30   
                                                                                                         codellama/CodeLlama-7b-Instruct-hf                         30   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                 2025-02-06_06-54-21_ab73cc7 0           No Drafter (Autoregressive)                                30   
                                                                                                         bigcode/tiny_starcoder_py                                  30   
                                                                                                         codellama/CodeLlama-7b-Instruct-hf                         30   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                                                 2025-02-06_11-43-18_ab73cc7 0           No Drafter (Autoregressive)                                30   
                                                                                                         bigcode/tiny_starcoder_py                                  30   
                                                                                                         codellama/CodeLlama-7b-Instruct-hf                         30   
                                                                                                         deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B                  30   
                                         tau/scrolls             202

In [225]:
def get_df_for_latex(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    df.drop(columns=["target", "temperature", "example_id_nunique", "new_toks"], inplace=True)
    df["dataset_path"] = df["dataset_path"].str.split("/").str[-1]
    df["drafter"] = df["drafter"].str.split("/").str[-1]
    df.rename(columns={"submission_id": "Hardware", "dataset_path": "Dataset", "drafter": "Drafter", "ttft_ms": "TTFT (ms)", "tpot_ms": "TPOT (ms)", "out_toks_per_sec": "T/s", "speedup": "Speedup"}, inplace=True)
    df.set_index(["Dataset", "Hardware", "Drafter"], inplace=True)
    return df

df_summary_deepseek_qwen_14b_temperature_0 = get_df_for_latex(df_summary_deepseek_qwen_14b_temperature_0.copy())
df_summary_deepseek_qwen_14b_temperature_0

TTFT (ms)  \
Dataset          Hardware                    Drafter                                    
cnn_dailymail    2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)        176.8   
                                             DeepSeek-R1-Distill-Qwen-1.5B      287.5   
                                             vicuna-68m                         243.0   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)         66.9   
                                             DeepSeek-R1-Distill-Qwen-1.5B      114.1   
                                             vicuna-68m                          91.0   
openai_humaneval 2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)         91.2   
                                             tiny_starcoder_py                  113.5   
                                             CodeLlama-7b-Instruct-hf           257.3   
                                             DeepSeek-R1-Distill-Qwen-1.5B      292.5   
                 2025-02-06_06-29-56_17d3c65 No Drafter (Autoregressive)         91.4   
                                             tiny_starcoder_py                  113.2   
                                             CodeLlama-7b-Instruct-hf           255.9   
                                             DeepSeek-R1-Distill-Qwen-1.5B      292.5   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)         43.7   
                                             tiny_starcoder_py                   49.6   
                                             CodeLlama-7b-Instruct-hf           153.0   
                                             DeepSeek-R1-Distill-Qwen-1.5B      165.5   
                 2025-02-06_11-43-18_ab73cc7 No Drafter (Autoregressive)         43.9   
                                             tiny_starcoder_py                   52.2   
                                             CodeLlama-7b-Instruct-hf           153.3   
                                             DeepSeek-R1-Distill-Qwen-1.5B      166.9   
scrolls          2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)       1481.0   
                                             DeepSeek-R1-Distill-Qwen-1.5B     1665.4   
                                             vicuna-68m                        1566.8   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)        464.2   
                                             DeepSeek-R1-Distill-Qwen-1.5B      533.0   
                                             vicuna-68m                         484.6   

                                                                            TPOT (ms)  \
Dataset          Hardware                    Drafter                                    
cnn_dailymail    2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)         51.7   
                                             DeepSeek-R1-Distill-Qwen-1.5B       69.9   
                                             vicuna-68m                          36.2   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)         33.7   
                                             DeepSeek-R1-Distill-Qwen-1.5B       51.0   
                                             vicuna-68m                          26.4   
openai_humaneval 2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)         48.7   
                                             tiny_starcoder_py                   43.9   
                                             CodeLlama-7b-Instruct-hf            77.9   
                                             DeepSeek-R1-Distill-Qwen-1.5B       70.4   
                 2025-02-06_06-29-56_17d3c65 No Drafter (Autoregressive)         51.9   
                                             tiny_starcoder_py                   43.8   
                                             CodeLlama-7b-Instruct-hf            77.0   
                                             DeepSeek-R1-Distill-Qwen-

In [226]:
# For each multi-index (Dataset, Hardware), find the maximum Speedup
columns_to_bold_max = ["T/s", "Speedup"]
for col in df_summary_deepseek_qwen_14b_temperature_0.columns:
    g = df_summary_deepseek_qwen_14b_temperature_0.groupby(["Dataset", "Hardware"])[col]
    index_to_bold = g.idxmax() if col in columns_to_bold_max else g.idxmin()
    df_summary_deepseek_qwen_14b_temperature_0[col] = df_summary_deepseek_qwen_14b_temperature_0[col].astype(str)
    df_summary_deepseek_qwen_14b_temperature_0.loc[index_to_bold, col] = df_summary_deepseek_qwen_14b_temperature_0.loc[index_to_bold, col].apply(lambda x: "\\textbf{" + x + "}")
df_summary_deepseek_qwen_14b_temperature_0

TTFT (ms)  \
Dataset          Hardware                    Drafter                                          
cnn_dailymail    2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)     \textbf{176.8}   
                                             DeepSeek-R1-Distill-Qwen-1.5B            287.5   
                                             vicuna-68m                               243.0   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)      \textbf{66.9}   
                                             DeepSeek-R1-Distill-Qwen-1.5B            114.1   
                                             vicuna-68m                                91.0   
openai_humaneval 2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)      \textbf{91.2}   
                                             tiny_starcoder_py                        113.5   
                                             CodeLlama-7b-Instruct-hf                 257.3   
                                             DeepSeek-R1-Distill-Qwen-1.5B            292.5   
                 2025-02-06_06-29-56_17d3c65 No Drafter (Autoregressive)      \textbf{91.4}   
                                             tiny_starcoder_py                        113.2   
                                             CodeLlama-7b-Instruct-hf                 255.9   
                                             DeepSeek-R1-Distill-Qwen-1.5B            292.5   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)      \textbf{43.7}   
                                             tiny_starcoder_py                         49.6   
                                             CodeLlama-7b-Instruct-hf                 153.0   
                                             DeepSeek-R1-Distill-Qwen-1.5B            165.5   
                 2025-02-06_11-43-18_ab73cc7 No Drafter (Autoregressive)      \textbf{43.9}   
                                             tiny_starcoder_py                         52.2   
                                             CodeLlama-7b-Instruct-hf                 153.3   
                                             DeepSeek-R1-Distill-Qwen-1.5B            166.9   
scrolls          2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)    \textbf{1481.0}   
                                             DeepSeek-R1-Distill-Qwen-1.5B           1665.4   
                                             vicuna-68m                              1566.8   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)     \textbf{464.2}   
                                             DeepSeek-R1-Distill-Qwen-1.5B            533.0   
                                             vicuna-68m                               484.6   

                                                                                TPOT (ms)  \
Dataset          Hardware                    Drafter                                        
cnn_dailymail    2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)             51.7   
                                             DeepSeek-R1-Distill-Qwen-1.5B           69.9   
                                             vicuna-68m                     \textbf{36.2}   
                 2025-02-06_06-54-21_ab73cc7 No Drafter (Autoregressive)             33.7   
                                             DeepSeek-R1-Distill-Qwen-1.5B           51.0   
                                             vicuna-68m                     \textbf{26.4}   
openai_humaneval 2025-02-06_02-10-28_d8c80fe No Drafter (Autoregressive)             48.7   
                                             tiny_starcoder_py              \textbf{43.9}   
                                             CodeLlama-7b-Instruct-hf                77.9   
                                             DeepSeek-R1-Distill-Qwen-1.5B           70.4   
                 2025-02-06_06-29-56_17d3c65 No Drafter (Autoregressive)             51.9   
                      

In [227]:
import re


def get_latex_table(df: pd.DataFrame) -> str:
    table: str = df.to_latex(escape=True, float_format="%.1f")
    table = re.sub(r'\\textbackslash textbf\\\{([\d.]+)\\\}', r'\\textbf{\1}', table)
    return table

print(get_latex_table(df_summary_deepseek_qwen_14b_temperature_0))

\begin{tabular}{lllllll}
\toprule
 &  &  & TTFT (ms) & TPOT (ms) & T/s & Speedup \\
Dataset & Hardware & Drafter &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{cnn\_dailymail} & \multirow[t]{3}{*}{2025-02-06\_02-10-28\_d8c80fe} & No Drafter (Autoregressive) & \textbf{176.8} & 51.7 & 19.2 & 1.0 \\
 &  & DeepSeek-R1-Distill-Qwen-1.5B & 287.5 & 69.9 & 14.1 & 0.73 \\
 &  & vicuna-68m & 243.0 & \textbf{36.2} & \textbf{27.4} & \textbf{1.43} \\
\cline{2-7}
 & \multirow[t]{3}{*}{2025-02-06\_06-54-21\_ab73cc7} & No Drafter (Autoregressive) & \textbf{66.9} & 33.7 & 29.7 & 1.0 \\
 &  & DeepSeek-R1-Distill-Qwen-1.5B & 114.1 & 51.0 & 19.4 & 0.65 \\
 &  & vicuna-68m & 91.0 & \textbf{26.4} & \textbf{37.6} & \textbf{1.27} \\
\cline{1-7} \cline{2-7}
\multirow[t]{16}{*}{openai\_humaneval} & \multirow[t]{4}{*}{2025-02-06\_02-10-28\_d8c80fe} & No Drafter (Autoregressive) & \textbf{91.2} & 48.7 & 20.4 & 1.0 \\
 &  & tiny\_starcoder\_py & 113.5 & \textbf{43.9} & \textbf{22.4} & \textbf{1.1} \\
 &  & CodeLlama-7